## Solve an inverse tarffic problem over polynomials of degree at most d

## Optionally use a regularizer from the poly kernel

In [1]:
%run ../Python_files/util.py

No dicts found; please check load_dicts...


In [2]:
import json

In [3]:
# read in node-link incidence matrtix
with open('node_link_incidence_Sioux.json', 'r') as json_file:
    N_dict = json.load(json_file)
    
N = zload('node_link_incidence_Sioux.pkz')

In [4]:
N.shape

(24, 76)

In [5]:
# bpr cost
def g_true(t):
    return 1 + 0.15 * (t ** 4)

In [6]:
# define polynomial kernel with degree d
def kernel(x, y, c, d):
    return (c + x*y) ** d

In [7]:
capac_list = []
free_flow_time_list = []
capac_dict = {}
free_flow_time_dict = {}

with open('SiouxFalls_net.txt', 'r') as f:
    read_data = f.readlines()

for row in read_data:
    if len(row.split()) == 11:
        key = row.split()[0] + ',' + row.split()[1]
        capac_list.append(float(row.split()[2]))
        free_flow_time_list.append(float(row.split()[3]))
        capac_dict[key] = float(row.split()[2])
        free_flow_time_dict[key] = float(row.split()[3])

In [8]:
numNode = N.shape[0]
numLink = N.shape[1]
assert(numLink == len(capac_list))

In [9]:
flow_list = []
flow_dict = {}

with open('SiouxFallsFlow.txt', 'r') as f:
    read_data = f.readlines()

for row in read_data:
    if len(row.split()) == 4:
        key = row.split()[0] + ',' + row.split()[1]
        flow_list.append(float(row.split()[2]))
        flow_dict[key] = float(row.split()[2])
#         print(row.split())

In [10]:
flow_normalized = [flow_list[i]/capac_list[i] for i in range(numLink)]

In [11]:
# construct kernel matrix; take d=5 for instance
c = 1.0
d = 4

phi = np.zeros((numLink, numLink))
for i in range(numLink):
    for j in range(numLink):
        phi[i, j] = kernel(flow_normalized[i], flow_normalized[j], c, d)

In [12]:
phi.shape

(76, 76)

In [13]:
np.linalg.matrix_rank(phi)

5

In [14]:
# read in demand data
with open('demands_Sioux.json', 'r') as json_file:
    demands = json.load(json_file)

In [15]:
od_list = []
for i in range(numNode + 1)[1:]:
    for j in range(numNode + 1)[1:]:
        if i != j:
            key = '(' + str(i) + ',' + str(j) + ')'
            od_list.append(key)

In [16]:
N.shape

(24, 76)

In [17]:
N_t = np.transpose(N)

In [18]:
N_t.shape

(76, 24)

In [19]:
model = Model("InverseVI_Sioux")

alpha = {}
for i in range(numLink):
    key = str(i)
    alpha[key] = model.addVar(name='alpha_' + key)
    
epsilon = model.addVar(name='epsilon')

yw = {}
for od in od_list:
    for i in range(numNode):
        key = od + str(i)
        yw[key] = model.addVar(name='yw_' + key)
        
model.update()

In [20]:
N.shape

(24, 76)

In [21]:
# add dual feasibility constraints
for od in od_list:
    for a in range(numLink):
        model.addConstr(sum([N[i, a] * yw[od+str(i)] for i in range(numNode)]) <= 
                        free_flow_time_list[a] * sum([alpha[str(j)] * phi[j, a] for j in range(numLink)]))
        
model.update()

In [22]:
# add increasing constraints
myList = flow_normalized
flow_sorted_idx = sorted(range(len(myList)),key=lambda x:myList[x])
for i in range(numLink):
    if (i < numLink-1):
        a_i_1 = flow_sorted_idx[i]
        a_i_2 = flow_sorted_idx[i+1]
        model.addConstr(sum([alpha[str(j)] * phi[j, a_i_1] for j in range(numLink)]) <= 
                        sum([alpha[str(j)] * phi[j, a_i_2] for j in range(numLink)]))
model.update()

In [23]:
model.addConstr(epsilon >= 0)

model.update()

In [24]:
# str(int(od.split(',')[0].split('(')[1])-1), str(int(od.split(',')[1].split(')')[0])-1)

In [25]:
# add primal-dual gap constraint
primal_cost = sum([free_flow_time_list[a] * sum([alpha[str(j)] * phi[j, a] for j in range(numLink)]) 
                   for a in range(numLink)])
dual_cost = sum([demands[od] * (yw[od + str(int(od.split(',')[1].split(')')[0])-1)] - 
                                yw[od + str(int(od.split(',')[0].split('(')[1])-1)]) 
                 for od in od_list])

model.addConstr(primal_cost - dual_cost <= epsilon)
model.addConstr(dual_cost - primal_cost <= epsilon)
        
model.update()

In [26]:
# add normalization constraint
model.addConstr(sum([alpha[str(j)] * phi[j, 0] for j in range(numLink)]) == 1)
total_cost = 0
for i in range(numLink):
    total_cost += sum([alpha[str(i)] * kernel(flow_normalized[i], flow_normalized[j], c, d) 
                       for j in range(numLink)])
        
model.addConstr(total_cost == sum([g_true(flow_normalized[i]) for i in range(numLink)]))
# model.addConstr(sum([alpha[str(i)] * kernel(flow_normalized[i], 0, c, d) for i in range(numLink)]) == 1)
model.update()

In [32]:
phi

array([[    1.12601279,     1.26344557,     1.12672844, ...,
            3.61383226,     3.46589973,     2.59037394],
       [    1.26344557,     1.57549796,     1.26500562, ...,
            9.53415887,     8.92738905,     5.58150333],
       [    1.12672844,     1.26500562,     1.12744832, ...,
            3.63545817,     3.48606397,     2.60231664],
       ..., 
       [    3.61383226,     9.53415887,     3.63545817, ...,
         1103.76079813,   972.03031773,   367.65861546],
       [    3.46589973,     8.92738905,     3.48606397, ...,
          972.03031773,   856.77079311,   326.56468055],
       [    2.59037394,     5.58150333,     2.60231664, ...,
          367.65861546,   326.56468055,   133.0823474 ]])

In [27]:
# Set objective
obj = 0

gama = 1e-8

for i in range(numLink):
    obj += sum([alpha[str(i)] * phi[i, j] * alpha[str(j)] for j in range(numLink)])

obj += gama * epsilon

model.setObjective(obj)

In [28]:
model.optimize()

Optimize a model with 42033 rows, 13325 columns and 3280299 nonzeros
Model has 2926 quadratic objective terms
Coefficient statistics:
  Matrix range    [7e-04, 2e+05]
  Objective range [1e-08, 1e-08]
  Bounds range    [0e+00, 0e+00]
  RHS range       [1e+00, 2e+02]
Presolve removed 76 rows and 0 columns
Presolve time: 0.62s
Presolved: 41957 rows, 13325 columns, 3274750 nonzeros
Presolved model has 2926 quadratic objective terms
Ordering time: 0.02s

Barrier statistics:
 Dense cols : 76
 Free vars  : 6
 AA' NZ     : 3.420e+06
 Factor NZ  : 4.285e+06 (roughly 60 MBytes of memory)
 Factor Ops : 4.430e+08 (less than 1 second per iteration)
 Threads    : 4

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   3.63482120e+00 -1.01338185e+00  7.60e+04 1.00e+00  1.00e+06     1s
   1   2.09097935e+07 -2.09753314e+07  6.41e+04 8.39e+02  8.47e+05     1s
   2   1.31100353e+08 -1.31267535e+08  4.61e+04 5.99e+02  6.12e+0

In [29]:
alpha_list = []
for v in model.getVars():
    if 'alpha' in v.varName:
#         print('%s %g' % (v.varName, v.x))
        alpha_list.append(v.x)

GurobiError: Unable to retrieve attribute 'x'

In [ ]:
alpha_list

In [ ]:
from matplotlib.mlab import prctile
import matplotlib.pyplot as plt
import pylab
from pylab import *

In [ ]:
xs = linspace(0, 2, 20)

In [ ]:
xs

In [ ]:
zs_true = [g_true(t) for t in xs]

In [ ]:
def g_est(t):
    return sum([alpha_list[m] * kernel(flow_normalized[m], t, c, d) for m in range(numLink)])

In [ ]:
zs_est = [g_est(t) for t in xs]

In [ ]:
true, = plt.plot(xs, zs_true, "ro-")
est, = plt.plot(xs, zs_est, "bs-")

plt.legend([true, est], ["g_true", "g_est"])
plt.xlabel('Scaled Flow')
plt.ylabel('Scaled Cost')
# plt.title('Threshold ($\eta$) versus Number of samples ($n$)')
# pylab.xlim(np.amin(n_range) - 1, np.amax(n_range) + 1)
# pylab.ylim(0, 1)
grid("on")
savefig('fittedCostFunc.eps')
plt.show()